In [ ]:
import sys
from pathlib import Path

def setup_project_paths():
    """Configurar paths del proyecto"""
    current_path = Path.cwd()
    
    # Buscar la raíz del proyecto (donde está src/)
    project_root = current_path
    while not (project_root / "src").exists() and project_root != project_root.parent:
        project_root = project_root.parent
    
    src_path = project_root / "src"
    if src_path not in [Path(p) for p in sys.path]:
        sys.path.append(str(src_path))
    
    return project_root, src_path

project_root, src_path = setup_project_paths()
print("✅ Proyecto configurado:", project_root)

ModuleNotFoundError: No module named 'utils.config_manager'; 'utils' is not a package

In [22]:
from data.load_data import load_data

df = load_data()

print("Data overview: \n")
# ¿Qué vamos a hacer?
print(df.head(10))
print(df.shape)  # ¿Cuántos clientes y variables?
print(df.info()) # ¿Qué tipo de información tenemos?

print("Data types: \n")
print(df.dtypes)

print("Data missing values: \n")
print(df.isnull().sum()) #no hay missing values

print("Data duplicated values: \n")
print(df.duplicated().sum()) #no hay duplicated values

print("Contando nuestra variable objetivo: \n")
print(df['Churn'].value_counts())

print("Distribución de la variable objetivo: \n")
print(df['Churn'].value_counts(normalize=True)) #aprox un 26.5% de churn

df['tenure'].max()










Loading data...
Data loaded successfully
Data overview: 

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   
5  9305-CDSKC  Female              0      No         No       8          Yes   
6  1452-KIOVK    Male              0      No        Yes      22          Yes   
7  6713-OKOMC  Female              0      No         No      10           No   
8  7892-POOKP  Female              0     Yes         No      28          Yes   
9  6388-TABGU    Male              0      No        Yes      62          Yes   

      MultipleLines InternetService OnlineSecurity  ... Devic

np.int64(72)

In [20]:
print("Se van mas quienes pagan menos o mas:")
df.groupby('Churn')['MonthlyCharges'].mean()

print("Se van mas clientes que son mas antiguos:")
df.groupby('Churn')['tenure'].mean()

Se van mas quienes pagan menos o mas:
Se van mas clientes que son mas antiguos:


Churn
No     37.569965
Yes    17.979133
Name: tenure, dtype: float64

In [24]:
import pandas as pd

# Crear segmentos de antigüedad
df['tenure_group'] = pd.cut(df['tenure'], 
                           bins=[0, 12, 24, 72], 
                           labels=['Nuevos (<1 año)', 'Establecidos (1-2 años)', 'Veteranos (2+ años)'])

# Ver churn y precio por segmento
analysis = df.groupby('tenure_group').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,  # % de churn
    'MonthlyCharges': 'mean'
}).round(2)

print("📊 Análisis por segmento:")
print(analysis)

📊 Análisis por segmento:
                         Churn  MonthlyCharges
tenure_group                                  
Nuevos (<1 año)          47.68           56.17
Establecidos (1-2 años)  28.71           61.36
Veteranos (2+ años)      14.04           70.61


C:\Users\diego\AppData\Local\Temp\ipykernel_3836\2395711629.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  analysis = df.groupby('tenure_group').agg({


In [32]:
#ver cuantos servicios tienen los clientes con mas de 2 anos de antiguedad

col_services = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# ¿Cuántos servicios tiene cada grupo?
# Primero, identifiquemos qué columnas son servicios
service_columns = [col for col in df.columns if col in [
    'PhoneService', 'MultipleLines', 'InternetService', 
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]]

print("Servicios disponibles:", service_columns)

# Crear segmentos de antigüedad
df['tenure_group'] = pd.cut(df['tenure'], 
                           bins=[0, 12, 24, 72], 
                           labels=['Nuevos (<1 año)', 'Establecidos (1-2 años)', 'Veteranos (2+ años)'])

analysis = df.groupby('tenure_group').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,  # % de churn
    'PhoneService': lambda x: (x == 'Yes').mean() * 100,
    'MultipleLines': lambda x: (x == 'Yes').mean() * 100,
    'InternetService': lambda x: (x == 'Yes').mean() * 100,
    'OnlineSecurity': lambda x: (x == 'Yes').mean() * 100,
    'OnlineBackup': lambda x: (x == 'Yes').mean() * 100,
    'DeviceProtection': lambda x: (x == 'Yes').mean() * 100,
    'TechSupport': lambda x: (x == 'Yes').mean() * 100,
    'StreamingTV': lambda x: (x == 'Yes').mean() * 100,
    'StreamingMovies': lambda x: (x == 'Yes').mean() * 100
}).round(2)

print("Análisis por segmento:")
print(analysis)

Servicios disponibles: ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
Análisis por segmento:
                         Churn  PhoneService  MultipleLines  InternetService  \
tenure_group                                                                   
Nuevos (<1 año)          47.68         90.02          22.76              0.0   
Establecidos (1-2 años)  28.71         91.21          36.82              0.0   
Veteranos (2+ años)      14.04         90.27          54.66              0.0   

                         OnlineSecurity  OnlineBackup  DeviceProtection  \
tenure_group                                                              
Nuevos (<1 año)                   12.28         15.49             14.94   
Establecidos (1-2 años)           22.56         24.80             25.49   
Veteranos (2+ años)               39.58         47.85             47.80   

                     

C:\Users\diego\AppData\Local\Temp\ipykernel_3836\1422432276.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  analysis = df.groupby('tenure_group').agg({


In [29]:
col_services = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for service in col_services:
    print(f"{service} unique values:", df[service].unique())

PhoneService unique values: ['No' 'Yes']
MultipleLines unique values: ['No phone service' 'No' 'Yes']
InternetService unique values: ['DSL' 'Fiber optic' 'No']
OnlineSecurity unique values: ['No' 'Yes' 'No internet service']
OnlineBackup unique values: ['Yes' 'No' 'No internet service']
DeviceProtection unique values: ['No' 'Yes' 'No internet service']
TechSupport unique values: ['No' 'Yes' 'No internet service']
StreamingTV unique values: ['No' 'Yes' 'No internet service']
StreamingMovies unique values: ['No' 'Yes' 'No internet service']


In [39]:
df['total_services'] = (
    (df['PhoneService'] == 'Yes').astype(int) +
    (df['MultipleLines'] == 'Yes').astype(int) +
    (df['InternetService'] == 'Yes').astype(int) +
    (df['OnlineSecurity'] == 'Yes').astype(int) +
    (df['OnlineBackup'] == 'Yes').astype(int) +
    (df['DeviceProtection'] == 'Yes').astype(int) +
    (df['TechSupport'] == 'Yes').astype(int) +
    (df['StreamingTV'] == 'Yes').astype(int) +
    (df['StreamingMovies'] == 'Yes').astype(int)
)

analysis = df.groupby('total_services').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'MonthlyCharges': 'mean'
}).round(2)

print(analysis)

                Churn  MonthlyCharges
total_services                       
0               43.75           24.90
1               21.11           30.08
2               32.83           51.13
3               36.48           69.36
4               31.34           78.73
5               25.55           86.31
6               22.49           93.64
7               12.41           99.43
8                5.29          104.63


In [45]:
# # Algo como:
# pd.crosstab([df['tenure_group'], df['total_services']], df['Churn'])

# Agrupando por ambas variables
analysis = df.groupby(['tenure_group', 'total_services']).agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100
}).round(2)

print(analysis)

                                        Churn
tenure_group            total_services       
Nuevos (<1 año)         0               54.24
                        1               36.16
                        2               54.46
                        3               54.55
                        4               53.88
                        5               64.29
                        6               51.61
                        7               75.00
                        8                 NaN
Establecidos (1-2 años) 0               16.67
                        1                8.98
                        2               28.65
                        3               36.89
                        4               39.89
                        5               37.17
                        6               41.51
                        7               14.29
                        8                0.00
Veteranos (2+ años)     0               11.11
                        1         

C:\Users\diego\AppData\Local\Temp\ipykernel_3836\3124277229.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  analysis = df.groupby(['tenure_group', 'total_services']).agg({


In [50]:
df['Contract'].unique()

analysis = df.groupby(['Contract', 'tenure_group']).agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'MonthlyCharges': 'mean'
}).round(2)

print(analysis)

                                        Churn  MonthlyCharges
Contract       tenure_group                                  
Month-to-month Nuevos (<1 año)          51.35           58.22
               Establecidos (1-2 años)  37.72           69.31
               Veteranos (2+ años)      30.86           78.78
One year       Nuevos (<1 año)          10.57           35.93
               Establecidos (1-2 años)   8.12           44.88
               Veteranos (2+ años)      11.89           71.65
Two year       Nuevos (<1 año)           0.00           28.77
               Establecidos (1-2 años)   0.00           32.31
               Veteranos (2+ años)       3.12           63.76


C:\Users\diego\AppData\Local\Temp\ipykernel_3836\23637946.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  analysis = df.groupby(['Contract', 'tenure_group']).agg({


In [55]:
analysis = df.groupby('gender').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'total_services': 'mean',
    'MonthlyCharges': 'mean'
}).round(2)

print(analysis)

        Churn  total_services  MonthlyCharges
gender                                       
Female  26.92            3.40           65.20
Male    26.16            3.33           64.33


In [58]:

print("PaymentMethod \n")

# PaymentMethod
analysisPaymentMethod = df.groupby('PaymentMethod').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'total_services': 'mean',
    'MonthlyCharges': 'mean'
}).round(2)

print(analysisPaymentMethod)

print("SeniorCitizen \n")

# SeniorCitizen 
analysisSeniorCitizen = df.groupby('SeniorCitizen').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'total_services': 'mean',
    'MonthlyCharges': 'mean'
}).round(2)

print(analysisSeniorCitizen)

print("Partner/Dependents \n")
# Partner/Dependents
analysisPartner = df.groupby('Partner').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'total_services': 'mean',
    'MonthlyCharges': 'mean'
}).round(2)

print(analysisPartner)



PaymentMethod 

                           Churn  total_services  MonthlyCharges
PaymentMethod                                                   
Bank transfer (automatic)  16.71            3.85           67.19
Credit card (automatic)    15.24            3.88           66.51
Electronic check           45.29            3.47           76.26
Mailed check               19.11            2.26           43.92
SeniorCitizen 

               Churn  total_services  MonthlyCharges
SeniorCitizen                                       
0              23.61            3.28           61.85
1              41.68            3.81           79.82
Partner/Dependents 

         Churn  total_services  MonthlyCharges
Partner                                       
No       32.96            2.93           61.95
Yes      19.66            3.83           67.78


In [ ]:
analysisDependents = df.groupby('Dependents').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'total_services': 'mean',
    'MonthlyCharges': 'mean'
}).round(2)

print(analysisDependents)



            Churn  total_services  MonthlyCharges
Dependents                                       
No          31.28            3.33           67.00
Yes         15.45            3.43           59.52


In [60]:
analysisPaperlessBilling = df.groupby('PaperlessBilling').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'total_services': 'mean',
    'MonthlyCharges': 'mean'
}).round(2)

print(analysisPaperlessBilling)






                  Churn  total_services  MonthlyCharges
PaperlessBilling                                       
No                16.33            2.85           51.99
Yes               33.57            3.71           73.55


In [61]:
print(df.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... StreamingTV  \
0  No phone service             DSL             No  ...          No   
1                No             DSL            Yes  ...          No   
2                No             DSL            Yes  ...          No   
3  No phone service             DSL            Yes  ...          No   
4                No     Fiber optic             No  ...          No   

  StreamingMovies        Contract PaperlessBilling              PaymentMethod  \
0          

In [62]:
df.to_csv('../../data/processed/telecom_churn_with_features.csv', index=False)